# 淘宝母婴购物数据分析\n## Taobao Maternity Shopping Data Analysis\n\n**项目目标**: 通过数据分析，洞察母婴电商平台的用户行为、商品偏好和客户价值，为运营决策提供数据支持。\n\n---

## 1. 项目背景与业务意义\n\n### 1.1 业务背景\n\n母婴电商市场具有以下特点：\n- **市场潜力巨大**: 中国母婴市场规模超过3万亿，年增长率10%+\n- **用户生命周期长**: 从孕期到孩子6岁，持续6-7年的消费需求\n- **复购率高**: 母婴用品是刚需，用户粘性强\n- **竞争激烈**: 平台众多，精细化运营成为关键\n\n### 1.2 分析目标\n\n| 维度 | 目标 | 业务价值 |\n|------|------|----------|\n| **用户画像** | 了解目标用户特征 | 精准营销、商品选品 |\n| **商品分析** | 识别热销品类 | 库存管理、采购决策 |\n| **行为分析** | 洞察购买规律 | 运营节奏、促销时机 |\n| **客户价值** | RFM价值分层 | 差异化运营、客户维系 |\n\n### 1.3 数据来源\n- **数据集**: 阿里云天池数据集 #45\n- **数据时间**: 2012-2015年\n- **样本规模**: 1,000位用户，15,000条交易记录

## 2. 环境准备与数据加载\n\n### 2.1 导入必要的库\n\n**业务意义**: 数据分析需要专业的工具库支持，包括数据处理、可视化和统计分析。

In [ ]:
# 数据处理\nimport pandas as pd  # 表格数据处理\nimport numpy as np   # 数值计算\n\n# 数据可视化\nimport matplotlib.pyplot as plt  # 基础绘图\nimport seaborn as sns            # 统计可视化\n\n# 时间处理\nfrom datetime import datetime, timedelta\n\n# 文件路径管理\nfrom pathlib import Path\n\n# 忽略警告信息\nimport warnings\nwarnings.filterwarnings('ignore')\n\n# 设置中文字体（解决中文乱码问题）\nplt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'Arial Unicode MS']\nplt.rcParams['axes.unicode_minus'] = False\n\n# 设置可视化样式\nsns.set_style("whitegrid")\nplt.style.use('seaborn-v0_8-darkgrid')\n\nprint("环境准备完成！")

### 2.2 定义项目路径\n\n**业务意义**: 规范化文件路径管理，确保数据读取和保存的一致性，便于项目迁移和协作。

In [ ]:
# 项目路径配置\nPROJECT_DIR = Path(".")  # 当前目录\nDATA_RAW = PROJECT_DIR / "data" / "raw"        # 原始数据\nDATA_PROCESSED = PROJECT_DIR / "data" / "processed"  # 处理后数据\nDATA_OUTPUT = PROJECT_DIR / "data" / "output"        # 输出图表\n\n# 创建必要的目录（如果不存在）\nDATA_PROCESSED.mkdir(parents=True, exist_ok=True)\nDATA_OUTPUT.mkdir(parents=True, exist_ok=True)\n\nprint(f"项目目录: {PROJECT_DIR.absolute()}")\nprint(f"原始数据: {DATA_RAW}")\nprint(f"处理后数据: {DATA_PROCESSED}")\nprint(f"输出图表: {DATA_OUTPUT}")

### 2.3 加载原始数据\n\n**业务意义**: 了解数据集规模是分析的基础，帮助我们评估数据质量和确定分析方向。\n\n**数据表说明**:\n- `tianchi_mum_baby.csv`: 婴儿信息表（用户维度）\n- `tianchi_mum_baby_trade_history.csv`: 交易历史表（交易维度）

In [ ]:
# 加载婴儿信息表\n# 业务意义: 包含用户ID和婴儿的基本信息，用于用户画像分析\nbaby_df = pd.read_csv(DATA_RAW / "tianchi_mum_baby.csv")\n\n# 加载交易历史表\n# 业务意义: 包含所有交易记录，是分析购买行为的核心数据\ntrade_df = pd.read_csv(DATA_RAW / "tianchi_mum_baby_trade_history.csv")\n\n# 数据规模概览\nprint("=" * 60)\nprint("数据集规模概览")\nprint("=" * 60)\nprint(f"婴儿信息表: {len(baby_df):,} 条记录")\nprint(f"交易历史表: {len(trade_df):,} 条记录")\nprint(f"\n婴儿信息表字段:\n{baby_df.columns.tolist()}")\nprint(f"\n交易历史表字段:\n{trade_df.columns.tolist()}")\n\n# 查看数据样例\nprint("\n婴儿信息表样例:")\nprint(baby_df.head())\nprint("\n交易历史表样例:")\nprint(trade_df.head())

## 3. 数据清洗与预处理\n\n### 3.1 婴儿信息数据清洗\n\n**业务意义**: 数据质量直接影响分析结果。清洗包括日期格式转换、性别标签映射等，确保数据可用性。

In [ ]:
# 日期格式转换\n# 原始格式: YYYYMMDD (如: 20130312)\n# 转换后: 标准日期格式，便于时间计算和可视化\nbaby_df['birthday'] = pd.to_datetime(baby_df['birthday'], format='%Y%m%d')\n\n# 性别标签映射\n# 业务意义: 将数字编码转换为可读的文本标签，便于理解和展示\n# 0=女婴, 1=男婴, 2=未知\nbaby_df['gender_label'] = baby_df['gender'].map({0: '女婴', 1: '男婴', 2: '未知'})\n\nprint("婴儿信息数据清洗完成")\nprint(f"\n日期范围: {baby_df['birthday'].min().date()} 至 {baby_df['birthday'].max().date()}")\nprint(f"\n性别分布:\n{baby_df['gender_label'].value_counts()}")\n\n# 数据质量检查\nprint("\n数据质量检查:")\nprint(f"缺失值数量: {baby_df.isnull().sum().sum()}")\nprint(f"重复记录数: {baby_df.duplicated().sum()}")

### 3.2 交易数据清洗\n\n**业务意义**: 交易数据是分析的核心，需要提取时间特征并识别异常值。

In [ ]:
# 日期格式转换和特征提取\ntrade_df['day'] = pd.to_datetime(trade_df['day'], format='%Y%m%d')\ntrade_df['year'] = trade_df['day'].dt.year      # 提取年份，用于年度对比\ntrade_df['month'] = trade_df['day'].dt.month    # 提取月份，用于季节性分析\ntrade_df['year_month'] = trade_df['day'].dt.to_period('M')  # 年月组合，用于趋势分析\n\nprint("交易数据清洗完成")\nprint(f"\n交易日期范围: {trade_df['day'].min().date()} 至 {trade_df['day'].max().date()}")\nprint(f"时间跨度: {(trade_df['day'].max() - trade_df['day'].min()).days} 天")\nprint(f"\n年度分布:\n{trade_df['year'].value_counts().sort_index()}")

### 3.3 异常值处理\n\n**业务意义**: 极端异常值可能是数据录入错误，需要识别并处理以保证分析准确性。\n\n**处理策略**: 使用99分位数作为阈值，超过该值的记录可能是异常。

In [ ]:
# 识别异常值\n# 使用99分位数作为阈值，超过该值的可能是数据错误\nq99 = trade_df['buy_mount'].quantile(0.99)\noutliers = trade_df[trade_df['buy_mount'] > q99]\n\nprint("异常值检测结果:")\nprint(f"  99分位数: {q99}")\nprint(f"  异常记录数: {len(outliers)} ({len(outliers)/len(trade_df)*100:.2f}%)")\n\nif len(outliers) > 0:\n    print(f"\n异常值示例:\n{outliers[['user_id', 'buy_mount', 'day']].head()}")\n\n# 处理异常值：移除极端异常（购买量>100）\n# 业务判断: 母婴用品单次购买超过100件属于极端情况，可能是数据错误\ntrade_clean = trade_df[trade_df['buy_mount'] <= 100].copy()\nremoved = len(trade_df) - len(trade_clean)\nprint(f"\n清洗后记录数: {len(trade_clean):,} (移除 {removed} 条异常记录)")\nprint(f"保留比例: {len(trade_clean)/len(trade_df)*100:.2f}%")

### 3.4 数据关联\n\n**业务意义**: 将用户信息与交易记录关联，可以进行用户维度的深度分析，如年龄分群、RFM分析等。\n\n**关联策略**: 以 `user_id` 为关联键，使用 left join 保留所有交易记录。

In [ ]:
# 合并数据集\nmerged_df = trade_clean.merge(baby_df, on='user_id', how='left')\n\n# 计算购买时的婴儿年龄\n# 业务意义: 了解不同年龄段用户的购买行为差异\nmerged_df['baby_age_days'] = (merged_df['day'] - merged_df['birthday']).dt.days\nmerged_df['baby_age_months'] = (merged_df['baby_age_days'] / 30).astype(int)\n\n# 创建年龄分群\n# 业务意义: 将连续的年龄转换为离散的分群，便于分组对比分析\nage_bins = [0, 6, 12, 24, 36, 72]\nage_labels = ['0-6个月', '6-12个月', '1-2岁', '2-3岁', '3岁以上']\nmerged_df['age_group'] = pd.cut(\n    merged_df['baby_age_months'], \n    bins=age_bins, \n    labels=age_labels, \n    right=False\n)\n\nprint(f"数据关联完成，合并后记录数: {len(merged_df):,}")\nprint(f"\n年龄分群分布:\n{merged_df['age_group'].value_counts().sort_index()}")\n\n# 查看关联后的数据样例\nprint("\n关联后数据样例:")\nprint(merged_df[['user_id', 'day', 'buy_mount', 'gender_label', 'baby_age_months', 'age_group']].head())

### 3.5 保存清洗后的数据\n\n**业务意义**: 保存清洗后的数据，便于后续分析复用和结果追溯。

In [ ]:
# 保存处理后的数据\nmerged_df.to_csv(DATA_PROCESSED / "merged_data.csv", index=False)\nbaby_df.to_csv(DATA_PROCESSED / "baby_clean.csv", index=False)\ntrade_clean.to_csv(DATA_PROCESSED / "trade_clean.csv", index=False)\n\nprint("数据保存完成！")\nprint(f"\n保存位置: {DATA_PROCESSED}")\nprint(f"\n生成的文件:")\nfor f in DATA_PROCESSED.glob("*.csv"):\n    size = f.stat().st_size / 1024  # KB\n    print(f"  - {f.name} ({size:.1f} KB)")